<a href="https://colab.research.google.com/github/maxrgnt/pythdc2-project2/blob/master/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# Panel Data
import pandas as pd
# System folders
import os
from pathlib import Path
# Visualization
import plotly.graph_objs as go
import seaborn as sns
%matplotlib inline

In [23]:
url = 'https://raw.githubusercontent.com/maxrgnt/pythdc2-project2/master/data/unemployment.csv'
bls = pd.read_csv(url)
print(bls.shape)
bls.sample(1)

(322, 5)


,State,Year,Abrv,UnemploymentRate,LaborRate
184,California,2009,CA,0.125602,0.651781


In [24]:
url = 'https://raw.githubusercontent.com/maxrgnt/pythdc2-project2/master/data/pctChangeGDP.csv'
bea = pd.read_csv(url)
print(bea.shape)
bea.sample(1)

(308, 4)


,State,Abrv,Year,Value
281,Arizona,AZ,2017,4.9


In [25]:
# read in the data
url='https://raw.githubusercontent.com/maxrgnt/pythdc2-project2/master/data/borderCrossing.csv'
bts = pd.read_csv(url)
print(bts.shape)
bts.sample(1)

(114588, 8)


,Abrv,State,Longitude,Latitude,Border,Year,Measure,Value
49785,WA,Washington,-117.63,49.0,US-Canada Border,2008,Pedestrians,10


In [26]:
def checkPctChange(df):
  print(df[(df['Abrv']=='TX') & (df['Year']==2014)])
  print(df[(df['Abrv']=='TX') & (df['Year']==2015)])
  print(df[(df['Abrv']=='TX') & (df['Year']==2016)])

In [27]:
def calcPctChange(df, col, pctChange_id):
  for y in list(df['Year'].unique()):
    states = list(df['Abrv'].unique())
    for s in states:
      if y != df['Year'].min():
        t0 = df.loc[(df['Year'] == y-1) & (df['Abrv'] == s), col].tolist()[0]
        t1 = df.loc[(df['Year'] == y) & (df['Abrv'] == s), col].tolist()[0]
        df.loc[(df['Year']==y) & (df['Abrv']==s), pctChange_id] = ((t1/t0)-1)*100
      else:
        df.loc[(df['Year']==y) & (df['Abrv']==s), pctChange_id] = 'N/A'
  # set drop index for first year where pctChange not calculated
  dropIndex = df.loc[df[pctChange_id]=='N/A'].index
  df.drop(dropIndex, inplace=True)

In [28]:
bts2 = bts.groupby(['Year','Abrv','State','Border'])[['Value']].sum().reset_index()
bts2.sample(1)

,Year,Abrv,State,Border,Value
82,2001,VT,Vermont,US-Canada Border,3146653


In [29]:
bts2.rename(columns={'Border':'US_Border'}, inplace = True)
calcPctChange(bts2, 'Value', 'border')
checkPctChange(bts2)

     Year Abrv  State         US_Border     Value   border
263  2014   TX  Texas  US-Mexico Border  80334766  4.71749
     Year Abrv  State         US_Border     Value   border
277  2015   TX  Texas  US-Mexico Border  82726100  2.97671
     Year Abrv  State         US_Border     Value   border
291  2016   TX  Texas  US-Mexico Border  85714503  3.61241


In [30]:
bea.rename(columns={'Value':'gdp'}, inplace = True)
checkPctChange(bea)

     State Abrv  Year  gdp
249  Texas   TX  2014  4.2
     State Abrv  Year  gdp
263  Texas   TX  2015  0.2
     State Abrv  Year  gdp
277  Texas   TX  2016 -0.2


In [31]:
calcPctChange(bls, 'UnemploymentRate', 'unemp')
calcPctChange(bls, 'LaborRate', 'labor')
checkPctChange(bls)

     State  Year Abrv  UnemploymentRate  LaborRate    unemp     labor
263  Texas  2014   TX          0.053764    0.64867 -19.4988 -0.627148
     State  Year Abrv  UnemploymentRate  LaborRate    unemp    labor
277  Texas  2015   TX          0.046521   0.637932 -13.4719 -1.65538
     State  Year Abrv  UnemploymentRate  LaborRate    unemp      labor
291  Texas  2016   TX          0.048399   0.637718  4.03763 -0.0335446


In [32]:
bls_cols = ['Year','Abrv','State','unemp','labor']
bea_cols = ['Year','Abrv','State','gdp']
bts2_cols = ['Year','Abrv','State','US_Border','border']
merge_cols = ['Year','State','Abrv']
df = bls[bls_cols].merge(bea[bea_cols], how = 'left', left_on= merge_cols, right_on= merge_cols,sort=True)
df = df.merge(bts2[bts2_cols], how = 'left', left_on=merge_cols, right_on=merge_cols,sort=True)

In [33]:
df['Year'] = df['Year'].astype(int)
for col in ['border','gdp','unemp','labor']:
  df[col] = df[col].astype(float)

In [34]:
df_melt = df.melt(['Year','Abrv','State','US_Border'], var_name='PctChange', value_name='Values')

In [35]:
df_melt.sample(10)

,Year,Abrv,State,US_Border,PctChange,Values
894,1998,VT,Vermont,US-Canada Border,border,-6.825800
1016,2007,NM,New Mexico,US-Mexico Border,border,23.237799
1167,2018,MI,Michigan,US-Canada Border,border,0.160159
183,2011,AZ,Arizona,US-Mexico Border,unemp,-9.211838
755,2009,WA,Washington,US-Canada Border,gdp,-0.600000
499,2012,NY,New York,US-Canada Border,labor,0.399795
398,2005,MN,Minnesota,US-Canada Border,labor,-0.924093
342,2001,MN,Minnesota,US-Canada Border,labor,-0.181278
919,2000,NY,New York,US-Canada Border,border,0.131530
784,2012,AL,Alaska,US-Canada Border,gdp,2.500000


In [36]:
# save new data structure
df.to_csv(Path.joinpath(Path.cwd(),'data','master.csv'), index = False)

In [0]:
# sns.set(rc={'figure.figsize':(18,6)})
# sns.lineplot(x = 'Year', y = 'vals', hue = 'cols', ci=None, data = df_melt);
# sns.pairplot(df, vars=['border_pctChange','gdp_pctChange','unemp_pctChange'], kind='reg');

In [0]:
# mapbox_access_token = open("assets/mytoken.mapbox_token").read()

# fig = go.Figure(go.Scattermapbox(
#     lat=df['Latitude'],
#     lon=df['Longitude'],
#     mode='markers',
#     marker=go.scattermapbox.Marker(
#         size=20,
#         colorscale='Purples',
#         color=df['Value']
#     ),
#     text=df['Value']

# ))
# fig.update_layout(
#     autosize=True,
#     hovermode='closest',
#     mapbox=go.layout.Mapbox(
#         accesstoken=mapbox_access_token,
#         bearing=0,
#         center=go.layout.mapbox.Center(
#             lat=39.8283,
#             lon=-98.5795
#         ),
#         pitch=0,
#         zoom=3
#     ),
# )
# fig

In [0]:
# fig = go.Figure(data=go.Choropleth(
#     locations=df2['StateAbrv'], # Spatial coordinates
#     z = df2['Value'].astype(float), # Data to be color-coded
#     locationmode = 'USA-states', # set of locations match entries in `locations`
#     colorscale = 'Purples',
#     colorbar_title = "Pedestrians",
# ))

# fig.update_layout(
#     title_text = 'Migration',
#     geo_scope='usa', # limite map scope to USA
# )

# fig.show()